In [11]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from time import time

In [75]:
movies_df = pd.read_csv('joined_movies_w_factors_and_genomes.csv')
print(movies_df.columns.values)
print(movies_df.dtypes)
movies_df

['title_script' 'genre_script' 'filename' ..., 'G1126' 'G1127' 'G1128']
title_script     object
genre_script     object
filename         object
title_p          object
movieId           int64
title            object
genres           object
X1              float64
X2              float64
X3              float64
X4              float64
X5              float64
X6              float64
X7              float64
X8              float64
X9              float64
X10             float64
X11             float64
X12             float64
X13             float64
X14             float64
X15             float64
X16             float64
X17             float64
X18             float64
X19             float64
X20             float64
X21             float64
X22             float64
X23             float64
                 ...   
G1099           float64
G1100           float64
G1101           float64
G1102           float64
G1103           float64
G1104           float64
G1105           float64
G1106           

,title_script,genre_script,filename,title_p,movieId,title,genres,X1,X2,X3,...,G1119,G1120,G1121,G1122,G1123,G1124,G1125,G1126,G1127,G1128
0,10 Things I Hate About You,"Comedy,Romance",10ThingsIHateAboutYou.txt,10 things i hate about you,2572,10 Things I Hate About You (1999),Comedy|Romance,-0.016455,0.018789,-0.013277,...,0.03600,0.02150,0.02275,0.01975,0.26425,0.09425,0.01425,0.01475,0.08425,0.01900
1,12 Years a Slave,Drama,12YearsaSlave.txt,12 years a slave,105844,12 Years a Slave (2013),Drama,0.008645,0.003144,0.001322,...,0.07650,0.04550,0.08550,0.06875,0.16950,0.11200,0.02500,0.05175,0.09950,0.02600
2,127 Hours,"Adventure,Drama,Thriller",127Hours.txt,127 hours,81562,127 Hours (2010),Adventure|Drama|Thriller,0.012059,-0.001237,-0.005137,...,0.06000,0.02250,0.01625,0.11225,0.16725,0.10250,0.02800,0.01700,0.18200,0.03775
3,1492: Conquest of Paradise,"Adventure,Drama",1492ConquestofParadise.txt,1492: conquest of paradise,8905,1492: Conquest of Paradise (1992),Adventure|Drama,-0.001974,-0.005688,-0.007619,...,0.09000,0.04075,0.02200,0.02625,0.21350,0.05575,0.02325,0.02800,0.07675,0.01575
4,15 Minutes,"Action,Crime,Thriller",15Minutes.txt,15 minutes,4167,15 Minutes (2001),Thriller,-0.011999,-0.005283,-0.017477,...,0.04275,0.01425,0.01225,0.03300,0.33225,0.04800,0.02475,0.01300,0.10300,0.01825
5,17 Again,"Comedy,Drama,Romance",17Again.txt,17 again,68135,17 Again (2009),Comedy|Drama,-0.009621,0.009796,0.000850,...,0.02575,0.01700,0.01575,0.01975,0.23350,0.05800,0.01500,0.00725,0.05775,0.01350
6,187,Drama,187.txt,187,1609,187 (One Eight Seven) (1997),Drama|Thriller,-0.004879,-0.009280,-0.012055,...,0.04350,0.01600,0.00975,0.05325,0.23750,0.08475,0.02025,0.00900,0.07475,0.01625
7,2001: A Space Odyssey,"Adventure,Sci-Fi",2001ASpaceOdyssey.txt,2001: a space odyssey,924,2001: A Space Odyssey (1968),Adventure|Drama|Sci-Fi,0.040611,-0.009755,0.003340,...,0.04100,0.02650,0.02225,0.07375,0.16975,0.10350,0.08850,0.01375,0.05700,0.01875
8,2012,"Action,Adventure,Drama,Sci-Fi,Thriller",2012.txt,2012,72378,2012 (2009),Action|Drama|Sci-Fi|Thriller,-0.021496,0.002300,0.003894,...,0.04725,0.01900,0.01825,0.01425,0.26750,0.08300,0.02050,0.01175,0.10775,0.01800
9,25th Hour,"Crime,Drama",25thHour.txt,25th hour,5954,25th Hour (2002),Crime|Drama,0.020300,-0.008028,-0.020965,...,0.10450,0.02650,0.01925,0.17400,0.33475,0.21325,0.03325,0.01850,0.06600,0.02000


For some reason there are still movies without scripts. Remove them here.

In [76]:
movies_df = movies_df[pd.notnull(movies_df["filename"])]
movies_df = movies_df[movies_df["filename"].str.contains(r"\.txt$")]
movies_df = movies_df[(movies_df["filename"] != "Apollo13.txt") & (movies_df["filename"] != "ScaryMovie2.txt")]
movies_df.reset_index(inplace=True)

In [77]:
script_files = ["raw/" + file for file in movies_df["filename"]]

Some stuff on CountVectorizer from the documentation:

"Convert a collection of text documents to a matrix of token counts"

Makes everything lowercase by default.

Default tokenizer: "The default regexp select tokens of 2 or more alphanumeric characters (punctuation is completely ignored and always treated as a token separator)."

Can use a built-in stop word list for English.

It cannot be parallelized, but you can use HashingVectorizer instead, which has some limitations.

In [78]:
vectorizer = CountVectorizer(input='filename',decode_error='ignore',stop_words='english', max_features=20000)

In [79]:
start = time()
features = vectorizer.fit_transform(script_files)
print(time()-start)

23.245896100997925


In [93]:
print(vectorizer.vocabulary_["great"])
print(movies_df.ix[500]["title"])
#great is 7778
#movie 500 is man on the moon
print(features[500,7778])
#we can verify that man of the moon has 17 "great"s

7778
Man on the Moon (1999)
17
